In [157]:
import folium
import osmnx as ox
from sodapy import Socrata
from math import log2
import random
import pandas as pd
from PIL import Image
import io

STOPS = '4vt2-8zrq'
TRIPS = 'ctwr-tvrd'
SCHEDULE = '4fvt-p2se'

EDMONTON_BLUE = '#035086'
COLORS=['#E21E26', #Red
        '#EE8C24', #Orange
        '#F8DB44', #Yellow
        '#107547', #Dark Green
        '#68C079', #Dark Green
        '#2D63AF', #Dark Blue
        '#5CBDDE', #Light Blue 
        '#10262F', #Black
        '#99479A', #Purple 
        '#EC9195', #Pink
        '#84999A', #Grey
        '#D9EBD ', #Muted Green
        ]

In [158]:
def random_hex_color():
    return random.choice(COLORS)

In [159]:
client = Socrata("data.edmonton.ca", None)

In [160]:
def get_stop(stop_id, client):
    return client.get(STOPS, stop_id=stop_id, limit=1)[0]

def get_stops_by_trip_id(trip_id, client):
    return client.get(SCHEDULE, trip_id=trip_id)

def get_trips_by_route_id(route_id, client):
    return client.get(TRIPS, route_id=route_id)

def get_trips_by_trip_id(trip_id, client):
    return client.get(TRIPS, trip_id=trip_id)

def get_routes_by_stop_id(stop_id, client):
    results = client.get(SCHEDULE, stop_id=stop_id)
    result_df = pd.DataFrame.from_records(results)
    routes = result_df['route_id'].unique()
    return routes 

def add_points(points, m, color=random_hex_color()):
    folium.PolyLine(locations=points, color=color).add_to(m)

In [174]:
def draw_route(route_id, client, m):
    color = random_hex_color()
    print(f'drawing {route_id} in {color}')
    trip0 = client.get(TRIPS, route_id=route_id, direction_id=0, limit=1)
    trip1 = client.get(TRIPS, route_id=route_id, direction_id=1, limit=1)

    if len(trip0):
        trip0=trip0[0]
        points0 = trip0['geometry_line']['coordinates'][0]
        points0 = [[point[1], point[0]] for point in points0]
        add_points(points0, m, color=color)
        stops0 = get_stops_by_trip_id(trip0['trip_id'], client)
        for stop in stops0:
            s = get_stop(stop['stop_id'], client)
            pos = [s['geometry_point']['coordinates'][1],s['geometry_point']['coordinates'][0]]
            folium.Circle(location=pos, color=EDMONTON_BLUE, fill_opacity=1, fill_color=EDMONTON_BLUE, radius=4).add_to(m)
    if len(trip1):
        trip1=trip1[0]
        points1 = trip1['geometry_line']['coordinates'][0]
        points1 = [[point[1], point[0]] for point in points1]
        add_points(points1, m, color=color)

        stops1 = get_stops_by_trip_id(trip1['trip_id'], client)

        for stop in stops1:
            s = get_stop(stop['stop_id'], client)
            pos = [s['geometry_point']['coordinates'][1],s['geometry_point']['coordinates'][0]]
            folium.Circle(location=pos, color=EDMONTON_BLUE, fill_opacity=1, fill_color=EDMONTON_BLUE, radius=4).add_to(m)

In [178]:
def draw_stop(stop_id, client):
    m = folium.Map()
    routes = get_routes_by_stop_id(stop_id, client)
    for route in routes:
        draw_route(route, client, m)
    
    stop = get_stop(stop_id, client)
    stop_location = [stop['geometry_point']['coordinates'][1], stop['geometry_point']['coordinates'][0]]


    folium.Marker(stop_location).add_to(m)
    return m

stop_id = 'S8108'

draw_stop(stop_id, client)

drawing 450 in #84999A
drawing 432 in #10262F
drawing 431 in #5CBDDE
drawing 430 in #10262F
drawing 433 in #68C079


In [163]:
stop_id = 'S8108'

stop = get_stop(stop_id, client)

trips = get_trips_by_stop_id(stop_id, client)



folium.Marker([stop['stop_lat'], stop['stop_lon']]).add_to(m)



m